In [1]:
import requests
import json
from tqdm import tqdm
import time

예시코드 listing_id = '6261834'

In [2]:
url = "https://www.airbnb.co.kr/api/v2/reviews"
params = {
    'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
    'currency':'KRW',
    'locale':'ko',
    'listing_id':'6261834',
    'role':'guest',
    '_format':'for_p3',
    '_limit':'100', # 몇개의 게시물을 보여줄 것인가(100개 이하로 설정가능)
    '_offset':'0',# 몇번째 게시물부터 시작할 것인가 0부터 시작
    '_order':'language_country'}

##### https://www.airbnb.co.kr/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&currency=KRW&locale=ko&listing_id=6261834&role=guest&_format=for_p3&_limit=7&_offset=7&_order=language_country

In [3]:
response = requests.get(url,params=params).text

In [4]:
result = json.loads(response)

- result:{'reviews', 'metadata'}
    -  reviews:[100 items]: {'collection_tag', __'comments'__, 'created_at', __'id'__, 'language', 'localized_date', 'rating', 'response', 'reviewee', __'reviewer'__}
    - metadata:{'reviews_count', 'should_show_review_translations'}

In [5]:
for k,v in result['reviews'][0].items():
     print("|| %30s || : %s" % (k,v)) # (|| key || : value) 형태로 출력

||                 collection_tag || : None
||                       comments || : 너무 깨끗하고 좋았습니다.
||                     created_at || : 2018-10-29T07:04:45Z
||                             id || : 342585456
||                       language || : ko
||                 localized_date || : 2018년 10월
||                         rating || : 5
||                       response || : 
||                       reviewee || : {'deleted': False, 'first_name': '현아', 'host_name': '현아', 'id': 24468677, 'is_superhost': False, 'picture_url': 'https://a0.muscache.com/im/pictures/5af5aaac-5719-4708-8056-b32d76f23b44.jpg?aki_policy=profile_x_medium', 'profile_path': '/users/show/24468677'}
||                       reviewer || : {'deleted': False, 'first_name': 'Doohoon', 'host_name': 'Doohoon', 'id': 81515153, 'is_superhost': False, 'picture_url': 'https://a0.muscache.com/im/pictures/user/c9774fbd-6469-4db7-81a1-f374743fcad8.jpg?aki_policy=profile_x_medium', 'profile_path': '/users/show/81515153'}


### listing_id 이용한 리뷰데이터 크롤링(서울)

In [2]:
import requests
import json
from tqdm import tqdm
import time

In [3]:
pwd

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\aribnb-review-crawler'

In [4]:
# 14610개 숙소 정보담은 home_jsn_314610,json 파일 불러오자, 경로설정은 각자하시길 
with open('C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\airbnb-data\\home_jsn_14610.json','r') as fp: 
    home_jsn = json.load(fp)

14610개 숙소의 모든 리뷰를 긁어오자

In [116]:
listing_ids = list(home_jsn.keys())

전역변수

In [203]:
url = "https://www.airbnb.co.kr/api/v2/reviews"
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

params = {
        'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
#         'currency':'KRW',
#         'locale':'ko',
#         'listing_id':
        'role':'guest',
        '_format':'for_p3',
#         '_limit':'100', # 몇개의 게시물을 보여줄 것인가(100개 이하로 설정가능)
#         '_offset':'0',# 몇번째 게시물부터 시작할 것인가 0부터 시작
#         '_order':'recent'
            }

review_jsn ={} # 리뷰데이터를 json으로 저장하기위해 우선 dict 생성
review_cnt_total = 0

listing_id 이용해 숙소의 메타데이터 중 리뷰 수를 가져오는 함수

In [204]:
def get_review_count(listing_id,params=params,url=url,headers=headers)->int:
    params_local = params.copy()
    review_jsn[listing_id] = [] # listing_id 키로 갖는 리뷰리스트 쌓자
    params_local['listing_id'] = listing_id # 리스팅 id를 해당 listing_id로 설정
    listing_id_res = requests.get(url, params=params_local, headers=headers).json() # 메타데이터 따기위한 request
    
    review_count= listing_id_res['metadata']['reviews_count'] # response하는 리뷰개수가 제한적(100개)이므로 리뷰수 따라 조정필요
    
    global review_cnt_total 
    review_cnt_total += review_count # 전역변수 review_cnt_total에 합산
    
    del params_local
    return review_count
    

listing_id와 review_count 이용해 리뷰를 review_jsn에 저장하는 함수

In [205]:
def get_reviews(listing_id,review_count,params=params,url=url,headers=headers):
    params_local = params.copy()
    params_local['listing_id'] = listing_id
    params_local['_limit'] = 100 # 최대 100개 리뷰 보겠다
    review_count_iter = review_count//100 # 100개단위로 리뷰개수 끊어줌 -> loop 돌아갈 횟수
    review_count_rest = review_count%100 # 100개 미만 나머지 리뷰의 수 -> loop 이후 가져올 나머지 리뷰 
    
    # loop 시작
    for i in range(review_count_iter): # 리뷰개수 100개 단위로 반복
#         print('iter:',i)
        params_local['_offset'] = 100*i

        print(params_local)
        res_100_reviews = requests.get(url, params=params_local, headers=headers).json()
        review_jsn[listing_id].extend(res_100_reviews['reviews']) # listing_id 를 key로 100개의 리뷰데이터를 쌓음
    
    # loop 끝 -> 나머지 리뷰 가져오기
    if review_count_rest >0:
        params_local['_limit'] = review_count_rest # 100개 미만 나머지 리뷰에 대해
        params_local['_offset'] = 100*review_count_iter

        print(params_local)
        res_rest_reviews = requests.get(url, params=params_local, headers=headers).json()
        review_jsn[listing_id].extend(res_rest_reviews['reviews'])
        del params_local

메인


In [206]:
params

{'_format': 'for_p3',
 'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20',
 'role': 'guest'}

In [207]:
if __name__=='__main__':
    # 전역변수 초기화
    review_jsn ={} 
    review_cnt_total = 0
    except_cnt = 0
    
    # 반복시작
    for listing_id in tqdm(listing_ids):
        retry = 2
        while retry >0:
            try:
                review_count = get_review_count(listing_id)
                get_reviews(listing_id,review_count)
                break
            except Exception as e:
                except_cnt += 1
                print(e.args,e.with_traceback)
                retry -= 1
                if listing_id in review_jsn.keys():
                    del review_jsn[listing_id]
                if retry == 0:
                    print("lising_id {} not stored".format(listing_id))
                    break
                continue
                
        
#         print('For this iter');print('# of review:', review_count);print('# of review stored:',len(review_jsn[listing_id]))



















  0%|                                                                                        | 0/14610 [00:00<?, ?it/s]

{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'role': 'guest', '_format': 'for_p3', 'listing_id': '18551259', '_limit': 69, '_offset': 0}




















  0%|                                                                              | 1/14610 [00:01<7:20:03,  1.81s/it]

{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'role': 'guest', '_format': 'for_p3', 'listing_id': '21841919', '_limit': 28, '_offset': 0}




















  0%|                                                                              | 2/14610 [00:03<7:48:38,  1.92s/it]

...

{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'role': 'guest', '_format': 'for_p3', 'listing_id': '21649537', '_limit': 1, '_offset': 0}




















  3%|██▌                                                                         | 495/14610 [11:50<5:37:35,  1.44s/it]

















  3%|██▌                                                                         | 496/14610 [11:50<5:37:10,  1.43s/it]

{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'role': 'guest', '_format': 'for_p3', 'listing_id': '25828438', '_limit': 6, '_offset': 0}




















  3%|██▌                                                                         | 497/14610 [11:52<5:37:20,  1.43s/it]

KeyboardInterrupt: 

In [215]:
len(review_jsn.keys())

498

In [214]:
review_cnt_total

7611

In [209]:
for k,v, in review_jsn.items():
    print(k, len(v))
    print(len(set([el['id'] for el in v])))

18551259 69
69
21841919 28
28
18518487 25
25
20835996 45
45
26311383 20
20
24399870 6
6
26165629 19
19
26260479 15
15
24535421 10
10
28422744 11
11
19914834 64
64
20679600 57
57
29509115 5
5
23439260 9
9
25064428 12
12
26038284 24
24
29502644 2
2
25065353 14
14
24159676 16
16
24901478 9
9
14715698 64
64
25064858 2
2
25291145 4
4
24901028 11
11
23751460 23
23
677424 191
191
13082759 66
66
25983022 4
4
10644346 49
49
13902596 46
46
21794339 32
32
20552862 67
67
26710519 18
18
15076829 78
78
14310194 53
53
1254326 77
77
10577796 60
60
20836209 47
47
19738793 84
84
1093295 117
117
26710859 25
25
20584786 59
59
15077442 51
51
14310332 28
28
26705129 29
29
14310466 41
41
23403825 28
28
8008577 0
0
20877202 41
41
19914927 55
55
14310134 48
48
15077441 38
38
26710028 23
23
14310098 47
47
19738905 67
67
1375433 48
48
14743122 27
27
24213426 12
12
9303687 26
26
19914733 49
49
19914890 59
59
19735702 52
52
19735808 61
61
23404634 19
19
26709697 22
22
21493428 23
23
25064108 4
4
24901655 8
8
23844

In [66]:



for listing_id in tqdm(listing_ids): # 숙소id listing_id 대해
    review_jsn[listing_id] = [] # listing_id 키로 갖는 리뷰리스트 쌓자
    
    params['listing_id'] = listing_id # 리스팅 id를 해당 listing_id로 설정
    
#     retry_cnt = 2 # 최대 2번동안 listing_id에 해당하는 response 요청하겠다 
#     while retry_cnt !=0:
#         try:
    listing_id_res = requests.get(url, params=params, headers=headers).json() # 메타데이터 따기위한 request
#             break
#         except Exception as e: # response 못받은 경우 retry
#             print('retry in listing_id!!',e)
#             retry_cnt -= 1
#             time.sleep(1)
#             continue
            
    review_count= listing_id_res['metadata']['reviews_count'] # response하는 리뷰개수가 제한적(100개)이므로 리뷰수 따라 조정필요
    review_cnt_total += review_count
    
    print(review_count)
    review_count_iter = review_count//100 # 100개단위로 리뷰개수 끊어줌
    review_count_rest = review_count%100 # 100개 미만 나머지 리뷰의 수
    
    print(review_count_iter,review_count_rest,'* = ',(review_count_iter)*100 + review_count_rest)
    
    for i in range(review_count_iter): # 리뷰개수 100개 단위로 반복
        print('iter:',i)
        params['_offset'] = 100*i
#         retry_cnt = 2
        
#         while retry_cnt !=0: # 최대 2번 동안 request
#             try:
        print(params)
        res_100_reviews = requests.get(url, params=params, headers=headers).json()
        review_jsn[listing_id].extend(res_100_reviews['reviews']) # listing_id 를 key로 100개의 리뷰데이터를 쌓음
#                 break
#             except Exception as e: # response 못받은 경우 retry
#                 print('retry in review!!',e)
#                 retry_cnt -= 1
#                 time.sleep(1)
#                 continue
        
     
        
        
            
    params['_limit'] = review_count_rest # 100개 미만 나머지 리뷰에 대해
    params['_offset'] = 100*review_count_iter
#     while retry_cnt !=0: # 최대 2번 동안 request
#             try:
    
    print(params)
    
    res_rest_reviews = requests.get(url, params=params, headers=headers).json()
    review_jsn[listing_id].extend(res_rest_reviews['reviews'])
#                 break
#             except Exception as e: # response 못받은 경우 retry
#                 print('retry in review!!',e)
#                 retry_cnt -= 1
#                 time.sleep(1)
#                 continue







  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

204
2 4 * =  204
iter: 0
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': '100', '_order': 'recent', 'listing_id': '4186309', '_offset': 0}
iter: 1
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': '100', '_order': 'recent', 'listing_id': '4186309', '_offset': 100}
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 4, '_order': 'recent', 'listing_id': '4186309', '_offset': 200}








 33%|████████████████████████████                                                        | 1/3 [00:06<00:12,  6.37s/it]

200
2 0 * =  200
iter: 0
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 4, '_order': 'recent', 'listing_id': '8008311', '_offset': 0}
iter: 1
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 4, '_order': 'recent', 'listing_id': '8008311', '_offset': 100}
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 0, '_order': 'recent', 'listing_id': '8008311', '_offset': 200}








 67%|████████████████████████████████████████████████████████                            | 2/3 [00:09<00:04,  4.51s/it]

223
2 23 * =  223
iter: 0
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 0, '_order': 'recent', 'listing_id': '1043588', '_offset': 0}
iter: 1
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 0, '_order': 'recent', 'listing_id': '1043588', '_offset': 100}
{'key': 'd306zoyjsyarp7ifhu67rjxn52tv0t20', 'currency': 'KRW', 'locale': 'ko', 'role': 'guest', '_format': 'for_p3', '_limit': 23, '_order': 'recent', 'listing_id': '1043588', '_offset': 200}








100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.34s/it]







결과를 json 파일로 저장하자

In [10]:
with open('review_seoul.json','w') as fp:
    json.dump(review_jsn,fp,ensure_ascii=False)

불러오자

- review_dict: {listing_id: [review_lst]} 형태이다

In [11]:
with open('review_seoul.json','r') as fp:
    review_dict = json.load(fp)

In [13]:
review_dict['18551259'][0]

{'collection_tag': None,
 'comments': '새벽이 도착했음에도 편안히 잘 쉴수 있었던 공간이었습니다. 반갑게 맞아준 헨리도 기억에 남네요. 너무 늦게 들어가서 놀아줄 시간이 없어 아쉬웠어요^^',
 'created_at': '2018-10-06T06:15:10Z',
 'id': 332696014,
 'language': 'ko',
 'localized_date': '2018년 10월',
 'rating': 5,
 'response': '',
 'reviewee': {'deleted': False,
  'first_name': 'David',
  'host_name': 'David',
  'id': 76010772,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/pictures/user/09c053d6-07a4-4c0e-a864-7fce8458baf6.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/76010772'},
 'reviewer': {'deleted': False,
  'first_name': 'Junghee',
  'host_name': 'Junghee',
  'id': 111507566,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/pictures/c1936b91-a6f7-4702-9db6-48df6ce827f9.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/111507566'}}